## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Isla Vista,US,34.4133,-119.8610,73.35,85,0,1.01,clear sky
1,1,Mar Del Plata,AR,-38.0023,-57.5575,62.62,87,0,25.01,clear sky
2,2,Fortuna,US,40.5982,-124.1573,52.36,89,100,2.15,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,81.70,68,41,11.95,scattered clouds
4,4,Hami,CN,42.8000,93.4500,72.43,14,56,6.96,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Butaritari,KI,3.0707,172.7902,81.70,68,41,11.95,scattered clouds
13,13,Hithadhoo,MV,-0.6000,73.0833,82.44,69,11,7.61,few clouds
16,16,Boa Vista,BR,2.8197,-60.6733,85.98,62,75,4.61,broken clouds
19,19,Maningrida,AU,-12.0667,134.2667,76.87,87,100,1.28,overcast clouds
22,22,Palembang,ID,-2.9167,104.7458,78.89,83,20,2.30,few clouds
23,23,Ribeira Grande,PT,38.5167,-28.7000,75.60,88,75,16.11,light intensity shower rain
37,37,Fukue,JP,32.6881,128.8419,80.47,85,100,24.94,light rain
44,44,Gat,IL,31.6100,34.7642,85.41,100,0,14.79,clear sky
54,54,Praya,CV,14.9215,-23.5087,84.74,70,20,14.97,few clouds
55,55,Cabedelo,BR,-6.9811,-34.8339,83.07,64,74,13.53,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                234
City                   234
Country                233
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Current Description    234
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.70,scattered clouds,3.0707,172.7902,
13,Hithadhoo,MV,82.44,few clouds,-0.6000,73.0833,
16,Boa Vista,BR,85.98,broken clouds,2.8197,-60.6733,
19,Maningrida,AU,76.87,overcast clouds,-12.0667,134.2667,
22,Palembang,ID,78.89,few clouds,-2.9167,104.7458,
23,Ribeira Grande,PT,75.60,light intensity shower rain,38.5167,-28.7000,
37,Fukue,JP,80.47,light rain,32.6881,128.8419,
44,Gat,IL,85.41,clear sky,31.6100,34.7642,
54,Praya,CV,84.74,few clouds,14.9215,-23.5087,
55,Cabedelo,BR,83.07,broken clouds,-6.9811,-34.8339,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.70,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
13,Hithadhoo,MV,82.44,few clouds,-0.6000,73.0833,Scoop Guest House
16,Boa Vista,BR,85.98,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
19,Maningrida,AU,76.87,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
22,Palembang,ID,78.89,few clouds,-2.9167,104.7458,ASTON Palembang Hotel & Conference Center
...,...,...,...,...,...,...,...
659,Belize,BZ,81.36,overcast clouds,17.2500,-88.7500,Villa San Juan
660,Languyan,PH,80.33,overcast clouds,5.2618,120.0807,
662,Huixtla,MX,80.28,overcast clouds,15.1500,-92.4667,Rocky Point Sonoran Sky
663,Aksu,CN,78.13,overcast clouds,41.1231,80.2644,Pudong Holiday Hotel


In [25]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna(subset=['Hotel Name'])
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.70,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
13,Hithadhoo,MV,82.44,few clouds,-0.6000,73.0833,Scoop Guest House
16,Boa Vista,BR,85.98,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
19,Maningrida,AU,76.87,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge
22,Palembang,ID,78.89,few clouds,-2.9167,104.7458,ASTON Palembang Hotel & Conference Center
...,...,...,...,...,...,...,...
650,Bac Lieu,VN,79.86,broken clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
659,Belize,BZ,81.36,overcast clouds,17.2500,-88.7500,Villa San Juan
662,Huixtla,MX,80.28,overcast clouds,15.1500,-92.4667,Rocky Point Sonoran Sky
663,Aksu,CN,78.13,overcast clouds,41.1231,80.2644,Pudong Holiday Hotel


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [56]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))